In [33]:
import requests
import json
url = "https://ap-south-1.aws.data.mongodb-api.com/app/data-uoruijs/endpoint/data/v1/action/findOne"

payload = json.dumps({
    "collection": "apikeylist",
    "database": "apis",
    "dataSource": "apikeys",
    "projection": {
        "_id": 1
    }
})

headers = {
  'Content-Type': 'application/json',
  'Access-Control-Request-Headers': '*',
  'api-key': 'L5m3Cs7rgirSzSOP2IK76aXxf6hR7RUVjqfHIu7yvMpiKvXHXDONNBXhDzHJ8Voo',
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"error":"endpoint 'GET /data/v1/action/findOne' not found","error_code":"EndpointNotFound","link":"https://services.cloud.mongodb.com/groups/659aebd6f638d1473060277d/apps/6652ae15654db90b15296fbc/logs?co_id=6652b5714c50af3479beb56e"}


In [42]:
import requests
import json

url = "https://ap-south-1.aws.data.mongodb-api.com/app/data-uoruijs/endpoint/data/v1/action/find"

payload = json.dumps({
    "collection": "apikeylist",
    "database": "apis",
    "dataSource": "apikeys",
    "limit": 5  
})

headers = {
    'Content-Type': 'application/json',
    'Access-Control-Request-Headers': '*',
    'api-key': 'L5m3Cs7rgirSzSOP2IK76aXxf6hR7RUVjqfHIu7yvMpiKvXHXDONNBXhDzHJ8Voo',
}

response = requests.post(url, headers=headers, data=payload)

print("Status Code:", response.status_code)
print("Response Text:", response.text)

try:
    response_json = response.json()
    print("Response JSON:", json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    print("Response is not in JSON format")


Status Code: 200
Response Text: {"documents":[]}
Response JSON: {
  "documents": []
}


In [12]:
from flask import Flask, render_template, request
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import threading
from helper import create_file_with_size, send_email  

In [16]:
def send_emails(sender_email, sender_password, receiver_email, subject, body, times, file_size, attachment, file_path):
    try:
        threads = []
        for time in range(1, times + 1):
            if times > 1:
                current_subject = f"{subject} ({time}/{times})"
                current_body = f"{body} ({time}/{times})"
            else:
                current_subject = subject
                current_body = body

            # Create a thread for each email sending task
            thread = threading.Thread(target=send_email, args=(sender_email, sender_password, receiver_email, current_subject, current_body, attachment))
            threads.append(thread)
            thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        os.remove(file_path)  # Clean up the created file after sending emails
        print(f"All emails sent successfully! {times} times. Total file size: {file_size*times} MB")
    except Exception as e:
        print(f"Error: {str(e)}")


In [21]:
import gradio as gr

def sendit(sender_email, sender_password, receiver_email, subject, body, times, file_size):
    file_path = create_file_with_size(int(file_size))
    with open(file_path, 'rb') as f:
        attachment = f.read()
    
    thread = threading.Thread(target=send_emails, args=(sender_email, sender_password, receiver_email, subject, body, int(times),file_size, attachment, file_path))
    thread.start()

    return "Email sending in progress. You will be notified once completed."

demo = gr.Interface(
    fn=sendit,
    inputs=["text", "text", "text", "text", "text", "text", "slider"],
    outputs=["text"]
)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
IMPORTANT: You are using gradio version 4.12.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://5518f1dd6b0a7bd0a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
